In [5]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=FutureWarning)



import pickle
import numpy as np
import eikon as ek
import pandas as pd
import datetime as dt
from tqdm import tqdm
import refinitiv.data as rd
from refinitiv.data.content import news
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [2]:
constituents = pd.read_csv("constituents.csv")
rics = constituents["RIC"].tolist()
rics = rics[:50]

In [3]:
rd.open_session()

analyzer = SentimentIntensityAnalyzer()

all_sentiment = {}

for ric in tqdm(rics):
    try:
        headlines = news.headlines.Definition(query=f"RIC:{ric}", count=15000).get_data()
        df = headlines.data.df.reset_index()

        df["sentiment"] = df["headline"].apply(lambda t: analyzer.polarity_scores(t)["compound"])

        df["versionCreated"] = pd.to_datetime(df["versionCreated"])
        df["date"] = df["versionCreated"].dt.date

        daily_sent = df.groupby("date")["sentiment"].mean()

        all_sentiment[ric] = daily_sent

    except Exception as e:
        print(f"Error fetching {ric}: {e}")

sentiment_df = pd.DataFrame(all_sentiment)

sentiment_df.index = pd.to_datetime(sentiment_df.index)
sentiment_df = sentiment_df.sort_index()



sentiment_df.to_csv('sentiment.csv')

100%|██████████| 50/50 [1:44:41<00:00, 125.62s/it]


In [6]:
this_year=dt.datetime.now().year

start=dt.datetime(this_year-1,8,29)
# today 
end=dt.datetime(this_year,11,28)

# reformat 
start=start.strftime('%Y-%m-%d')
end=end.strftime('%Y-%m-%d')

In [14]:
sent = pd.read_csv("sentiment.csv")

RIC = sent.columns.tolist()

vars = ['TR.PriceClose','TR.NumOfStrongBuy','TR.NumOfBuy',
        'TR.NumOfStrongSell','TR.NumOfSell']

query_list=[]
for var in vars:
    query_list.append(f'{var}(Frq=D,SDate={start},EDate={end}).date')
    query_list.append(f'{var}(Frq=D,SDate={start},EDate={end})')
query_list

query_list=['TR.PriceClose(Frq=D,SDate=2024-08-29,EDate=2025-11-28).date',
 'TR.PriceClose(Frq=D,SDate=2024-08-29,Curn=EUR,EDate=2025-11-28)',
 'TR.NumOfStrongBuy(Frq=D,SDate=2024-08-29,EDate=2025-11-28).date',
 'TR.NumOfStrongBuy(Frq=D,SDate=2024-08-29,EDate=2025-11-28)',
 'TR.NumOfBuy(Frq=D,SDate=2024-08-29,EDate=2025-11-28).date',
 'TR.NumOfBuy(Frq=D,SDate=2024-08-29,EDate=2025-11-28)',
 'TR.NumOfStrongSell(Frq=D,SDate=2024-08-29,EDate=2025-11-28).date',
 'TR.NumOfStrongSell(Frq=D,SDate=2024-08-29,EDate=2025-11-28)',
 'TR.NumOfSell(Frq=D,SDate=2024-08-29,EDate=2025-11-28).date',
 'TR.NumOfSell(Frq=D,SDate=2024-08-29,EDate=2025-11-28)']

data_dic={}
rd.open_session()
data_dic[vars[0]]= rd.get_data(RIC, query_list[0:2]) # TR.PriceClose


rd.close_session()

In [17]:
# TR.NumOfStrongBuy
rd.open_session()
data_dic[vars[1]] = rd.get_data(RIC, query_list[2:4])
rd.close_session()

In [18]:
# TR.NumOfBuy
rd.open_session()
data_dic[vars[2]] = rd.get_data(RIC, query_list[4:6])
rd.close_session()

In [19]:
# TR.NumOfStrongSell
rd.open_session()
data_dic[vars[3]] = rd.get_data(RIC, query_list[6:8])
rd.close_session()

In [20]:
# TR.NumOfSell
rd.open_session()
data_dic[vars[4]] = rd.get_data(RIC, query_list[8:10])
rd.close_session()

In [21]:
for i in range(len(vars)):
    temp=data_dic[vars[i]].copy()
    temp=temp.drop_duplicates()
    temp.columns=['Instrument','Date','value']
    temp=temp.pivot(index='Date',columns='Instrument',values='value')
    data_dic[vars[i]]=temp

with open('data_dic.pkl','wb') as f:
    pickle.dump(data_dic,f)

In [22]:
with open('data_dic.pkl','rb') as f:
    hw2=pickle.load(f)

In [23]:
print(hw2.keys())

dict_keys(['TR.PriceClose', 'TR.NumOfStrongBuy', 'TR.NumOfBuy', 'TR.NumOfStrongSell', 'TR.NumOfSell'])


In [24]:
price_local=hw2[vars[0]]
strongbuy=hw2[vars[1]].ffill()
strongbuy=strongbuy.bfill()
buy=hw2[vars[2]].ffill()
buy=buy.bfill()
strongsell=hw2[vars[3]].ffill()
strongsell=strongsell.bfill()
sell=hw2[vars[4]].ffill()
sell=sell.bfill()

# rec1= strongbuy+buy-strongsell-sell
rec1= strongbuy-buy-strongsell+sell

In [25]:
hw3={'price':price_local,
      'rec':rec1}

In [31]:

price = hw3['price']
rec = hw3['rec']


In [33]:
price

Instrument,1COVG.DE,AALB.AS,ABB.ST,ABI.BR,ABNd.AS,ACCP.PA,ACKB.BR,AD.AS,ADP.PA,ADSGn.DE,...,AYV.PA,AZN.ST,BAKKA.OL,BAMN.AS,BAR.BR,BASFn.DE,BAYGn.DE,BEIG.DE,BESI.AS,DATE
Date,,,,,,,,,,,,,,,,,,,,,
NaT,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2020-04-01,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2021-01-26,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2024-08-29,55.4,35.04,51.567179,55.52,15.585,38.2,172.6,30.88,117.5,231.6,...,6.15,157.836185,47.851823,3.798,12.21,45.725,27.81,131.3,121.6,<NA>
2024-08-30,55.4,35.22,51.792883,55.3,15.49,38.07,179.8,31.06,118.5,232.0,...,6.29,157.310299,49.087149,3.774,12.3,45.85,27.865,130.75,117.8,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-11-24,59.54,26.9,60.028667,53.86,26.82,46.14,216.2,35.23,122.1,152.35,...,10.75,158.757634,38.478138,8.025,11.85,44.33,30.595,90.3,124.4,<NA>
2025-11-25,59.24,27.54,59.705028,54.5,28.55,47.01,218.8,35.47,123.4,154.2,...,10.83,160.876923,38.551524,8.14,12.04,44.71,30.83,92.14,124.6,<NA>
2025-11-26,58.52,27.66,60.958817,53.42,29.21,46.94,218.4,35.55,125.3,158.85,...,10.91,161.393647,38.913221,8.31,12.01,44.62,30.36,92.06,129.6,<NA>


In [37]:
price.index = pd.to_datetime(price.index)
price = price[price.index.notna()]
price = price.sort_index()

price = price.loc["2024-08-29" : "2025-11-28"]

In [36]:
price.index


DatetimeIndex(['2020-04-01', '2021-01-26', '2024-08-29', '2024-08-30',
               '2024-09-02', '2024-09-03', '2024-09-04', '2024-09-05',
               '2024-09-06', '2024-09-09',
               ...
               '2025-11-18', '2025-11-19', '2025-11-20', '2025-11-21',
               '2025-11-24', '2025-11-25', '2025-11-26', '2025-11-27',
               '2025-11-28',        'NaT'],
              dtype='datetime64[ns]', name='Date', length=325, freq=None)

In [38]:
price.to_csv('price.csv')

In [41]:
rec.index = pd.to_datetime(rec.index)
rec = rec[rec.index.notna()]
rec = rec.sort_index()

rec = rec.loc["2024-08-29" : "2025-11-28"]

rec

Instrument,1COVG.DE,AALB.AS,ABB.ST,ABI.BR,ABNd.AS,ACCP.PA,ACKB.BR,AD.AS,ADP.PA,ADSGn.DE,...,AYV.PA,AZN.ST,BAKKA.OL,BAMN.AS,BAR.BR,BASFn.DE,BAYGn.DE,BEIG.DE,BESI.AS,DATE
Date,,,,,,,,,,,,,,,,,,,,,
2024-08-30,0,-2,0,-4,1,-4,0,4,-1,-4,...,-3,0,1,-1,-4,-3,-1,-4,-6,<NA>
2024-09-02,0,-2,0,-4,1,-4,0,4,-1,-4,...,-3,0,1,-1,-4,-4,-1,-4,-7,<NA>
2024-09-03,0,-2,0,-4,1,-4,0,4,-1,-4,...,-3,0,1,-1,-4,-5,-1,-4,-7,<NA>
2024-09-04,0,-2,0,-4,1,-4,0,4,-1,-4,...,-3,0,1,-1,-4,-5,-1,-4,-7,<NA>
2024-09-05,0,-2,0,-4,1,-3,0,4,-1,-4,...,-3,0,1,-1,-4,-5,-1,-4,-7,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-11-17,-1,-1,-2,-7,-5,-3,0,0,-2,-3,...,-4,-3,-2,2,-3,5,0,1,-4,<NA>
2025-11-18,-1,-1,-2,-7,-5,-3,0,0,-2,-3,...,-4,-3,-2,2,-3,5,0,1,-4,<NA>
2025-11-20,-1,-1,-2,-7,-5,-3,0,0,-2,-3,...,-4,-3,-2,2,-3,5,0,1,-4,<NA>


In [42]:
rec.to_csv('recommendation.csv')